In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import render_sphere as rs
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

#For the sphere x = [theta, phi]

def norm(x,w):
    return w[0]**2 + np.sin(x[0])**2. * w[1]**2.

def metric(x,v,w):
    return w[0]*v[0] + np.sin(x[0])**2. * w[1]*v[1]

def co_vector_from_vector(x, w):
  #Here for the sphere :
  metric = [[1.,0.], [0., np.sin(x[0])**2.]]
  return np.matmul(metric, w)

def vector_from_co_vector(x, alpha):
    inversemetric = [[1., 0.],[0., 1./np.sin(x[0])**2]]
    return np.matmul(inversemetric, alpha)

def hamiltonian_equation(x, alpha):
  if (abs(np.sin(x[0])) < 1e-20):
    raise ValueError("Cannot handle the poles of the sphere")
  Fx = np.array([alpha[0], alpha[1]/np.sin(x[0])**2]) #this is g_{ab} alpha^b
  Falpha = np.array([np.cos(x[0])/np.sin(x[0])**3. * alpha[1]**2., 0.])
  return Fx, Falpha

def parallel_transport(x, alpha, w, number_of_time_steps):
    dimension = len(x) #Dimension of the manifold
    delta = 1./number_of_time_steps
    epsilon = delta

    #To store the computed values of trajectory and transport
    xtraj = np.zeros((number_of_time_steps+1, dimension))
    pwtraj = np.zeros((number_of_time_steps+1, dimension))
    alphatraj = np.zeros((number_of_time_steps+1, dimension))
    #initialisation
    xtraj[0] = x
    alphatraj[0] = alpha
    pwtraj[0] = w
    initialNorm = np.sqrt(norm(x,w))
    initialCrossProductWithVelocity = metric(x,v,w)
    RK_Steps = [0.5, 1]
    time  = 0.
    print("Lauching computation with epsilon :", epsilon, "delta :", delta, "number_of_time_steps : ", number_of_time_steps)

    for k in range(number_of_time_steps):
        time = time + delta
        xcurr = xtraj[k]
        alphacurr = alphatraj[k]

        #Compute the position of the next point on the geodesic
        for i,step in enumerate(RK_Steps):
            Fx, Falpha = hamiltonian_equation(xcurr, alphacurr)
            xcurr = xtraj[k] + step * delta * Fx
            alphacurr = alphatraj[k] + step * delta * Falpha

        #Co-vector of w_k : g^{ab} w_b
        betacurr = co_vector_from_vector(xtraj[k], pwtraj[k])
        perturbations = [1,-1]
        Weights = [0.5, -0.5]
        Jacobi = np.zeros(2)
        #For each perturbation, compute the perturbed geodesic
        for i, pert in enumerate(perturbations):
            alphaPk = alphatraj[k] + pert * epsilon * betacurr
            alphaPerturbed = alphaPk
            xPerturbed = xtraj[k]
            for step in RK_Steps:
                Fx, Falpha = hamiltonian_equation(xPerturbed, alphaPerturbed)
                xPerturbed = xtraj[k] + step * delta * Fx
                alphaPerturbed = alphaPk + step * delta * Falpha
            #Update the estimate
            Jacobi = Jacobi + Weights[i] * xPerturbed
        pwtraj[k+1] = Jacobi / (epsilon * delta)
        xtraj[k+1] = xcurr
        alphatraj[k+1] = alphacurr;
    return xtraj, alphatraj, pwtraj

def plot_norm(x, w):
    norms = [norm(x[i], w[i]) for i in range(len(xtraj))]
    plt.plot(norms)
    plt.show()


x = np.array([0.5,-1.])
x3D = rs.localChartTo3D(x)
direction = np.array([1,0.5])
v = np.pi * direction /np.sqrt(np.dot(direction,co_vector_from_vector(x,direction)))
alpha = co_vector_from_vector(x, v)
w = [alpha[1], -alpha[0]]

w3D = rs.chartVelocityTo3D(x, w)
v3D = rs.chartVelocityTo3D(x,v)

#true endpoint and parallel vector
n = np.linalg.norm(v3D)
x3DFinal = np.cos(n)*x3D + np.sin(n) * v3D/n
v3DFinal = -np.sin(n)* n * x3DFinal + np.cos(n) * v3D

proj = np.dot(v3D,w3D)/np.dot(v3D, v3D)
projOrtho = np.dot(np.cross(x3D,v3D), w3D)/np.dot(v3D,v3D)
truepw3D = proj * v3DFinal + projOrtho*np.cross(x3D, v3D)

nbSteps = 100
xtraj, alphatraj, pwtraj = parallel_transport(x, alpha, w, nbSteps)
pwtraj3D = rs.chartVelocityTo3D(xtraj[-1], pwtraj[-1])
print("Estimated : ", pwtraj3D, "Exact :", truepw3D)

rs.render_sphere(xtraj, pwtraj)






Lauching computation with epsilon : 0.01 delta : 0.01 number_of_time_steps :  100
Estimated :  [-1.15568977 -1.2373258  -0.24072329] Exact : [-1.06599684 -1.0506352  -0.1683259 ]
[[ 0.25903472]
 [ 0.27954843]
 [ 0.2997864 ]
 [ 0.31972864]
 [ 0.33935548]
 [ 0.35864753]
 [ 0.37758575]
 [ 0.39615143]
 [ 0.41432626]
 [ 0.43209228]
 [ 0.44943196]
 [ 0.46632818]
 [ 0.48276426]
 [ 0.49872397]
 [ 0.51419157]
 [ 0.52915177]
 [ 0.54358983]
 [ 0.55749147]
 [ 0.570843  ]
 [ 0.58363121]
 [ 0.5958435 ]
 [ 0.60746781]
 [ 0.61849267]
 [ 0.62890719]
 [ 0.6387011 ]
 [ 0.64786473]
 [ 0.65638903]
 [ 0.66426559]
 [ 0.67148664]
 [ 0.67804505]
 [ 0.68393435]
 [ 0.68914872]
 [ 0.69368302]
 [ 0.69753278]
 [ 0.70069418]
 [ 0.70316411]
 [ 0.70494014]
 [ 0.7060205 ]
 [ 0.70640414]
 [ 0.70609066]
 [ 0.70508039]
 [ 0.70337431]
 [ 0.7009741 ]
 [ 0.69788214]
 [ 0.69410147]
 [ 0.68963582]
 [ 0.6844896 ]
 [ 0.67866788]
 [ 0.6721764 ]
 [ 0.66502157]
 [ 0.65721045]
 [ 0.64875073]
 [ 0.63965076]
 [ 0.62991952]
 [ 0.619566